In [1]:
#!pip install ultralytics face_recognition

In [2]:
import cv2
from ultralytics import YOLO
import face_recognition

def main():
    # Inicializar modelos
    yolo_model = YOLO('yolov8n.pt')

    # Cargar personas conocidas
    known_face_encodings = []
    known_face_names = []

    # Agregar persona de referencia
    # Check if the reference image file exists before attempting to load it
    try:
        reference_image = face_recognition.load_image_file("JRE.jpg")
        # Use the HOG model for face detection which runs on CPU
        reference_encoding = face_recognition.face_encodings(reference_image, model="hog")
        if reference_encoding: # Ensure encoding was successful
            known_face_encodings.append(reference_encoding[0]) # Append the first encoding found
            known_face_names.append("Persona Objetivo")
        else:
            print("Could not find a face in the reference image.")
    except FileNotFoundError:
        print("Error: JRE.jpg not found. Please upload the reference image.")
        return # Exit if the reference image is not found
    except IndexError:
        print("Error processing reference image: No faces found or unexpected format.")
        return


    # Captura de video
    # Using 0 for the default camera, may need to change depending on setup
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return # Exit if camera cannot be opened

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame from video stream.")
            break

        # Detectar personas usando YOLO
        # Confidence threshold can be adjusted (conf=0.5 by default)
        results = yolo_model(frame, classes=[0], verbose=False) # classes=[0] for person

        for r in results:
            boxes = r.boxes
            for box in boxes:
                # Ensure box coordinates are integers
                x1, y1, x2, y2 = box.xyxy[0].int().tolist()

                # Extraer región de la persona, ensuring valid slice
                if y1 >= 0 and y2 <= frame.shape[0] and x1 >= 0 and x2 <= frame.shape[1]:
                    person_crop = frame[y1:y2, x1:x2]

                    # Reconocimiento facial usando el modelo HOG en la región detectada por YOLO
                    # Use the HOG model for face detection within the person crop
                    face_locations = face_recognition.face_locations(person_crop, model="hog")
                    face_encodings = face_recognition.face_encodings(person_crop, face_locations, model="hog")

                    name = "Desconocido"
                    # Compare detected faces within the crop to known faces
                    for face_encoding in face_encodings:
                        # Ensure known_face_encodings is not empty before comparing
                        if known_face_encodings:
                            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
                            if True in matches:
                                match_index = matches.index(True)
                                name = known_face_names[match_index]
                                break # Stop searching once a match is found

                    # Dibujar resultados en el frame original
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    # Adjust text position to be inside the bounding box if it's too close to the top
                    text_y_position = y1 - 10 if y1 - 10 > 10 else y1 + 10
                    cv2.putText(frame, name, (x1, text_y_position), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                else:
                    print(f"Warning: Invalid bounding box coordinates: ({x1}, {y1}, {x2}, {y2})")


        # Mostrar el frame
        # This will likely not work in a typical Colab environment without a display
        # Consider alternatives like saving frames or using a virtual display
        try:
            cv2.imshow('Person Identification', frame)
            # Check for key press to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except cv2.error as e:
            print(f"Error displaying frame: {e}")
            print("cv2.imshow() may not work in this environment. Consider saving frames or using a virtual display.")
            break # Exit loop if display fails

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

/home/hellraiser/miniforge3/envs/torch_env/lib/python3.11/site-packages/face_recognition_models/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Could not find a face in the reference image.


TypeError: compute_face_descriptor(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], face: _dlib_pybind11.full_object_detection, num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vector
    2. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], num_jitters: int = 0) -> _dlib_pybind11.vector
    3. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], faces: _dlib_pybind11.full_object_detections, num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vectors
    4. (self: _dlib_pybind11.face_recognition_model_v1, batch_img: list[numpy.ndarray[(rows,cols,3),numpy.uint8]], batch_faces: list[_dlib_pybind11.full_object_detections], num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vectorss
    5. (self: _dlib_pybind11.face_recognition_model_v